In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
df = pd.read_csv('data/flight.csv')
df = df.drop(['tailnum','dep_time','sched_dep_time','dep_delay'],axis=1)
print('Data Types:')
print(df.dtypes.value_counts())
print()
print(df.info())
df.head()

Data Types:
object     4
int64      3
float64    2
dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336776 entries, 0 to 336775
Data columns (total 9 columns):
arr_time          328063 non-null float64
sched_arr_time    336776 non-null int64
arr_delay         327346 non-null float64
carrier           336776 non-null object
flight            336776 non-null int64
origin            336776 non-null object
dest              336776 non-null object
distance          336776 non-null int64
time_hour         336776 non-null object
dtypes: float64(2), int64(3), object(4)
memory usage: 23.1+ MB
None


arr_time  sched_arr_time  arr_delay carrier  flight origin dest  distance  \
0     830.0             819       11.0      UA    1545    EWR  IAH      1400   
1     850.0             830       20.0      UA    1714    LGA  IAH      1416   
2     923.0             850       33.0      AA    1141    JFK  MIA      1089   
3    1004.0            1022      -18.0      B6     725    JFK  BQN      1576   
4     812.0             837      -25.0      DL     461    LGA  ATL       762   

             time_hour  
0  2013-01-01 05:00:00  
1  2013-01-01 05:00:00  
2  2013-01-01 05:00:00  
3  2013-01-01 05:00:00  
4  2013-01-01 06:00:00

In [3]:
pd.concat([df.isnull().sum(), 100 * df.isnull().sum()/len(df)], 
              axis=1).rename(columns={0:'Missing Records', 1:'Percentage (%)'})

Missing Records  Percentage (%)
arr_time                   8713        2.587180
sched_arr_time                0        0.000000
arr_delay                  9430        2.800081
carrier                       0        0.000000
flight                        0        0.000000
origin                        0        0.000000
dest                          0        0.000000
distance                      0        0.000000
time_hour                     0        0.000000

In [4]:
cols = ['arr_time', 'arr_delay']
df[cols] = df[cols].fillna(df[cols].median())
df['distance(miles)'] = round(df.distance.apply(lambda x: x*0.62137),2)
df.head()

arr_time  sched_arr_time  arr_delay carrier  flight origin dest  distance  \
0     830.0             819       11.0      UA    1545    EWR  IAH      1400   
1     850.0             830       20.0      UA    1714    LGA  IAH      1416   
2     923.0             850       33.0      AA    1141    JFK  MIA      1089   
3    1004.0            1022      -18.0      B6     725    JFK  BQN      1576   
4     812.0             837      -25.0      DL     461    LGA  ATL       762   

             time_hour  distance(miles)  
0  2013-01-01 05:00:00           869.92  
1  2013-01-01 05:00:00           879.86  
2  2013-01-01 05:00:00           676.67  
3  2013-01-01 05:00:00           979.28  
4  2013-01-01 06:00:00           473.48

In [5]:
df['time_hour'] = pd.to_datetime(df['time_hour'])
df['date'] = df['time_hour'].map(lambda x: x.strftime('%Y-%m-%d'))
df['month'] = df['time_hour'].dt.month
df.head()

arr_time  sched_arr_time  arr_delay carrier  flight origin dest  distance  \
0     830.0             819       11.0      UA    1545    EWR  IAH      1400   
1     850.0             830       20.0      UA    1714    LGA  IAH      1416   
2     923.0             850       33.0      AA    1141    JFK  MIA      1089   
3    1004.0            1022      -18.0      B6     725    JFK  BQN      1576   
4     812.0             837      -25.0      DL     461    LGA  ATL       762   

            time_hour  distance(miles)        date  month  
0 2013-01-01 05:00:00           869.92  2013-01-01      1  
1 2013-01-01 05:00:00           879.86  2013-01-01      1  
2 2013-01-01 05:00:00           676.67  2013-01-01      1  
3 2013-01-01 05:00:00           979.28  2013-01-01      1  
4 2013-01-01 06:00:00           473.48  2013-01-01      1

In [6]:
pd.crosstab(df['origin'], df['month'])

month     1     2      3      4      5      6      7      8     9      10  \
origin                                                                      
EWR     9893  9107  10420  10531  10592  10175  10475  10359  9550  10104   
JFK     9161  8421   9697   9218   9397   9472  10023   9983  8908   9143   
LGA     7950  7423   8717   8581   8807   8596   8927   8985  9116   9642   

month     11    12  
origin              
EWR     9707  9922  
JFK     8710  9146  
LGA     8851  9067

In [7]:
pd.pivot_table(df, values='arr_delay', index=['origin'],
                  columns=['month'], aggfunc=np.mean)

month          1         2         3          4         5          6   \
origin                                                                  
EWR     12.317699  7.970462  9.994434  13.598044  5.098659  15.920295   
JFK      1.278026  3.929343  2.424461   6.744413  2.026711  16.905089   
LGA      3.172579  2.714940  3.410692  11.520336  2.573294  13.759656   

month          7         8         9         10        11         12  
origin                                                                
EWR     14.778520  6.469254 -4.735288  2.530978  0.611723  18.368273  
JFK     19.521700  5.787338 -4.470251 -3.593241 -0.903674  12.246556  
LGA     13.070684  5.126322 -2.886463  0.118648  1.456446  11.245726

In [8]:
df['origin'].replace({'EWR':'ewr', 'JFK':'jfk', 'LGA':'lga'}, inplace=True)
df.loc[df.origin=='ewr', ['dest']] ='BAY'
df['flight'] = df['flight'].astype('float')
df.head()

arr_time  sched_arr_time  arr_delay carrier  flight origin dest  distance  \
0     830.0             819       11.0      UA  1545.0    ewr  BAY      1400   
1     850.0             830       20.0      UA  1714.0    lga  IAH      1416   
2     923.0             850       33.0      AA  1141.0    jfk  MIA      1089   
3    1004.0            1022      -18.0      B6   725.0    jfk  BQN      1576   
4     812.0             837      -25.0      DL   461.0    lga  ATL       762   

            time_hour  distance(miles)        date  month  
0 2013-01-01 05:00:00           869.92  2013-01-01      1  
1 2013-01-01 05:00:00           879.86  2013-01-01      1  
2 2013-01-01 05:00:00           676.67  2013-01-01      1  
3 2013-01-01 05:00:00           979.28  2013-01-01      1  
4 2013-01-01 06:00:00           473.48  2013-01-01      1

In [9]:
print(df.groupby('origin')['dest'].nunique().to_frame())
df['sum_arr_delay'] = df.groupby('carrier')['arr_delay'].transform(pd.Series.cumsum)
df[df.carrier=='UA'].head(10)

        dest
origin      
ewr        1
jfk       70
lga       68


arr_time  sched_arr_time  arr_delay carrier  flight origin dest  distance  \
0      830.0             819       11.0      UA  1545.0    ewr  BAY      1400   
1      850.0             830       20.0      UA  1714.0    lga  IAH      1416   
5      740.0             728       12.0      UA  1696.0    ewr  BAY       719   
12     924.0             917        7.0      UA   194.0    jfk  LAX      2475   
13     923.0             937      -14.0      UA  1124.0    ewr  BAY      2565   
16     854.0             902       -8.0      UA  1187.0    ewr  BAY      2227   
24     858.0             915      -17.0      UA  1077.0    ewr  BAY      1085   
26     945.0             931       14.0      UA   303.0    jfk  SFO      2586   
32     933.0             932        1.0      UA   496.0    lga  IAH      1416   
37    1016.0             947       29.0      UA  1665.0    ewr  BAY      2454   

             time_hour  distance(miles)        date  month  sum_arr_delay  
0  2013-01-01 05:00:00           869.92  2013-01-01      1           11.0  
1  2013-01-01 05:00:00           879.86  2013-01-01      1           31.0  
5  2013-01-01 05:00:00           446.77  2013-01-01      1           43.0  
12 2013-01-01 06:00:00          1537.89  2013-01-01      1           50.0  
13 2013-01-01 06:00:00          1593.81  2013-01-01      1           36.0  
16 2013-01-01 06:00:00          1383.79  2013-01-01      1           28.0  
24 2013-01-01 06:00:00           674.19  2013-01-01      1           11.0  
26 2013-01-01 06:00:00          1606.86  2013-01-01      1           25.0  
32 2013-01-01 06:00:00           879.86  2013-01-01      1           26.0  
37 2013-01-01 06:00:00          1524.84  2013-01-01      1           55.0

In [10]:
df1 = df[~np.isnan(df['arr_delay'])]
df2 = df.loc[:,~df.columns.duplicated()]
df3 = df.drop_duplicates(subset=['origin'], keep='first')
df4 = pd.concat([df, pd.get_dummies(df['origin'])], axis=1)
df5 = pd.concat([df, pd.get_dummies(df.select_dtypes(include='object'))], axis=1)

from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')